# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun

## Dataset

### Overview
The National Basketball Association (NBA) Games Data Kaggle dataset (https://www.kaggle.com/datasets/nathanlauga/nba-games) is a dataset of all NBA games from 2004-2020.  The information was collected from the NBA stats website and contains key information about the perfromance of each team during the game.  I will be using this data to train a model on what the key characteristics are for an NBA team in a win.  Using a data-driven approach, NBA teams can choose their focus area for strategizing and practicing when preparing for a successful season.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'nba_automl'

experiment=Experiment(ws, experiment_name)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
# Read Data
dataset = Dataset.get_by_name(ws, name='nba-games-data')
df = dataset.to_pandas_dataframe()

from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, test_size=0.15)

In [6]:
automl_settings = {"n_cross_validations": 3,
    "primary_metric": "AUC_weighted",
    "enable_early_stopping": True,
    "max_concurrent_iterations": 2,  
    "experiment_timeout_hours": 0.25
}

automl_config = AutoMLConfig( task="classification",
    debug_log="automl_errors.log",
    training_data=train_data,
    label_column_name='HOME_TEAM_WINS',
    **automl_settings)

In [7]:
remote_run = experiment.submit(automl_config)

2022-11-10:05:09:30,992 INFO     [font_manager.py:1337] generated new fontManager
2022-11-10:05:09:32,766 WARNING  [local_experiment_driver.py:145] Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


Experiment,Id,Type,Status,Details Page,Docs Page
nba_automl,AutoML_4fedc2bd-b32a-422b-b43d-8a9d288abd05,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


2022-11-10:05:27:47,802 INFO     [explanation_client.py:334] Using default datastore for uploads


## Run Details


In [8]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [16]:
best_auto_run, fitted_auto_model = remote_run.get_output()
best_auto_run.get_metrics()

{'average_precision_score_weighted': 0.9513611264927344,
 'average_precision_score_macro': 0.9487396074402269,
 'average_precision_score_micro': 0.9525692085445309,
 'recall_score_macro': 0.868559323455746,
 'f1_score_weighted': 0.8739619526735739,
 'f1_score_macro': 0.8696975478028808,
 'balanced_accuracy': 0.868559323455746,
 'recall_score_weighted': 0.8741675336650356,
 'log_loss': 0.28488757018134825,
 'norm_macro_recall': 0.7371186469114922,
 'recall_score_micro': 0.8741675336650356,
 'AUC_weighted': 0.9489711666046156,
 'AUC_macro': 0.9489711666046156,
 'precision_score_micro': 0.8741675336650356,
 'accuracy': 0.8741675336650356,
 'precision_score_weighted': 0.8738923790808917,
 'weighted_accuracy': 0.8794216158730547,
 'matthews_correlation': 0.7395354255481595,
 'f1_score_micro': 0.8741675336650356,
 'precision_score_macro': 0.8709801595582927,
 'AUC_micro': 0.9510218554990466,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_4fedc2bd-b32a-422b-b43d-8a9d288abd05

In [17]:
import joblib
joblib.dump(fitted_auto_model, 'nba-games-model.pkl')

['nba-games-model.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [20]:
from azureml.core import Model
model = Model.register(workspace=ws,
                       model_name='nba-games-model', 
                       model_path='./nba-games-model.pkl',
                       description='NBA games AutoML run.',
                       tags={'course': 'capstone'})

Registering model nba-games-model


In [25]:
best_auto_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')
best_auto_run.download_file('outputs/conda_env_v_1_0_0.yml', 'envFile.yml')

In [48]:
from azureml.core.model import InferenceConfig

inf_config = InferenceConfig(
    entry_script='score.py',
    environment=ws.environments['nba-2']
)

In [49]:
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1
)

service = Model.deploy(
    ws, "nba-games-deploy2", [model], inf_config, deployment_config
)

In [51]:
type(service)

azureml.core.webservice.aci.AciWebservice

TODO: In the cell below, send a request to the web service you deployed to test it.

In [50]:
import requests
headers = {'Content-type':'application/json'}
resp = requests.post(service.scoring_uri, test_data.to_json(), headers=headers)
print(resp.text)

MissingSchema: Invalid URL 'None': No scheme supplied. Perhaps you meant http://None?

TODO: In the cell below, print the logs of the web service and delete the service

In [39]:
service.get_logs()

2022-11-10:05:52:43,592 ERROR    [_azureml_exception.py:273] Received bad response from Model Management Service:
Response Code: 404
Headers: {'Date': 'Thu, 10 Nov 2022 05:52:43 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'x-ms-client-request-id': '36b7b7b7-13fd-4f9b-9780-c97c1a0b7d4d', 'x-ms-client-session-id': 'c68ca68d-043b-4d0d-822d-a55df8da382d', 'api-supported-versions': '1.0, 2018-03-01-preview, 2018-11-19', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload', 'X-Content-Type-Options': 'nosniff', 'x-aml-cluster': 'vienna-southcentralus-02', 'x-request-time': '0.840', 'Content-Encoding': 'gzip'}
Content: b'{"code":"NotFound","statusCode":404,"message":"The specified resource was not found.","details":[{"code":"ResourceNotFound","message":"The Resource \'Microsoft.ContainerInstance/containerGroups/nba-games-ws-PCybbNxSIkS2Yt8Nd_XKqg\' under resource group \'aml-quickstarts-

WebserviceException: WebserviceException:
	Message: Received bad response from Model Management Service:
Response Code: 404
Headers: {'Date': 'Thu, 10 Nov 2022 05:52:43 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'x-ms-client-request-id': '36b7b7b7-13fd-4f9b-9780-c97c1a0b7d4d', 'x-ms-client-session-id': 'c68ca68d-043b-4d0d-822d-a55df8da382d', 'api-supported-versions': '1.0, 2018-03-01-preview, 2018-11-19', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload', 'X-Content-Type-Options': 'nosniff', 'x-aml-cluster': 'vienna-southcentralus-02', 'x-request-time': '0.840', 'Content-Encoding': 'gzip'}
Content: b'{"code":"NotFound","statusCode":404,"message":"The specified resource was not found.","details":[{"code":"ResourceNotFound","message":"The Resource \'Microsoft.ContainerInstance/containerGroups/nba-games-ws-PCybbNxSIkS2Yt8Nd_XKqg\' under resource group \'aml-quickstarts-214827\' 

In [38]:
service.delete()

WebserviceException: WebserviceException:
	Message: There is a deployment operation in flight for the Service: nba-games-ws
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "There is a deployment operation in flight for the Service: nba-games-ws"
    }
}

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
